In [1]:
!pip install tensorflow-gpu==2.0.0b1

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [3]:
tf.__version__

'2.0.0-beta1'

In [4]:
# Google 드라이브를 마운트하려면 이 셀을 실행하세요.
from google.colab import drive
drive.mount('/content/drive')

E0708 11:15:18.546977 139721583073152 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d5df0069828e>", line 2, in <module>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.6/dist-packages/google/colab/drive.py", line 167, in mount
    u'Drive File Stream encountered a problem and has stopped',
  File "/usr/local/lib/python3.6/dist-packages/pexpect/spawnbase.py", line 341, in expect
    timeout, searchwindowsize, async_)
  File "/usr/local/lib/python3.6/dist-packages/pexpect/spawnbase.py", line 369, in expect_list
    return exp.expect_loop(timeout)
  File "/usr/local/lib/python3.6/dist-packages/pexpect/expect.py", line 111, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/usr/local/lib/python3.6/dist-packages/pexpect/pty_spawn.py", line 499, in read_nonblocking
    if (timeout != 0) and select(timeout):


KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
im_dir = os.path.join("/content/gdrive/My Drive/workspace/Python/ML_Basic/RNN/", 'data')

In [0]:
im_dir

In [0]:
x_train = []

In [0]:
file_list = os.listdir(im_dir)
file_list = sorted(file_list, key=str.lower)

In [0]:
import cv2

im = cv2.imread(os.path.join(im_dir, file_list[0]))
im.shape

In [0]:
import cv2

for name in file_list:
    print(os.path.join(im_dir, name))
    im = cv2.imread(os.path.join(im_dir, name))
    im = cv2.resize(im, dsize = (256, 256))
    im = im.astype(np.float32)/255.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    x_train.append(im)

    
print(os.path.join(im_dir, file_list[0]))
im = cv2.imread(os.path.join(im_dir, file_list[0]))
im = cv2.resize(im, dsize = (256, 256))
im = im.astype(np.float32)/255.
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
x_train.append(im)
    
x_train = np.array(x_train)    

In [0]:
x_train[0].dtype

In [0]:
plt.imshow(x_train[1])
plt.colorbar()
plt.show()

In [0]:
def create_model():
  inputs = layers.Input(shape=(256, 256, 3), name='inputs')
  e_inputs = layers.Input(shape=(1, 256, 256, 6), name='e_in')
  
  R00 = layers.ConvLSTM2D(12, 3, 1, 'same', return_sequences=True)(e_inputs)
  #print(R00.shape)
  R01 = layers.ConvLSTM2D(12, 3, 1, 'same', return_sequences=True)(R00)
  #print(R01.shape)
  R02 = layers.ConvLSTM2D(12, 3, 1, 'same')(R01)
  #print(R02.shape)
  Ahat0 = layers.Conv2D(3, 3, 1, 'same')(R02)
#   print(Ahat.shape)
#   print(inputs.shape)

  e0 = layers.Subtract()([inputs, Ahat0])
  e1 = layers.Subtract()([Ahat0, inputs])
  E0 = layers.concatenate([e0, e1], axis=-1)
  E0 = keras.activations.relu(E0)
  #print(E0.shape)
  
  A1 = layers.Conv2D(3, 3, 1, 'same')(E0)
  
  E0 = layers.Reshape((1, 256, 256, 6))(E0)
  R10 = layers.ConvLSTM2D(6, 3, 1, 'same', return_sequences=True)(E0)
  R11 = layers.ConvLSTM2D(6, 3, 1, 'same', return_sequences=True)(R10)
  R12 = layers.ConvLSTM2D(6, 3, 1, 'same')(R11)
  Ahat1 = layers.Conv2D(3, 3, 1, 'same', name='ahat')(R12)
  
  e2 = layers.Subtract()([A1, Ahat1])
  e3 = layers.Subtract()([Ahat1, A1])
  E1 = layers.concatenate([e2, e3], axis=-1)
  E1 = keras.activations.relu(E1)
  
  E0 = layers.Reshape((1, 256, 256, 6), name='E0')(E0)
  E1 = layers.Reshape((1, 256, 256, 6), name='E1')(E1)
  
  hM = keras.Model(inputs=[inputs, e_inputs], outputs=[E1])
  pM = keras.Model(inputs=[inputs, e_inputs], outputs=[Ahat1])
  
  return hM, pM

In [0]:
hm, pm = create_model()

In [0]:
hm.compile('adam', 'mean_squared_error')
pm.compile('adam', 'mean_squared_error')

In [0]:
hm.summary()

In [0]:
pm.summary()

In [0]:
import matplotlib.pyplot as plt

In [0]:
x_train = np.array(x_train)

In [0]:
def grad(aInput, aOutput, anE):
  e0 = np.subtract(aInput, aOutput)
  e1 = np.subtract(aOutput, aInput)
  E = np.concatenate([e0, e1], axis=-1)
  E = np.maximum(E, 0)
  E = np.expand_dims(E, 0)
  
  hm.fit({'inputs':aInput, 'e_in':anE}, [E], 1, 1)
  anE = hm.predict({'inputs':aInput, 'e_in':anE})
  pm.fit({'inputs': aInput, 'e_in':anE}, [aOutput], 1, 1)  
  pred_img = pm.predict({'inputs': aInput, 'e_in': anE})
  
  return pred_img

In [0]:
in_E = np.zeros((1, 1, 256, 256, 6))

for epoch in range(20):
  for i in range(len(x_train)-1):
    in_im = x_train[i].reshape((1, 256, 256, 3))
    ou_im = x_train[i+1].reshape((1, 256, 256, 3))
    
    next_img = grad(in_im, ou_im, in_E)
    
    fig, ax = plt.subplots(1, 3)    
    ax[0].set_axis_off()
    ax[0].imshow(in_im.reshape(256, 256, 3))
    ax[1].set_axis_off()
    ax[1].imshow(ou_im.reshape(256, 256, 3))
    ax[2].set_axis_off()
    ax[2].imshow(next_img.reshape(256, 256, 3))
    
    
    plt.show()
    
    #model.fit({'inputs':in_im, 'e_in':in_E}, [labels, labels, ou_im], 1, 1)
    #in_E = model.predict({'inputs':in_im, 'e_in':in_E})[0]
    #in_E = in_E.reshape((1, 1, 256, 256, 6))
    #final_E = in_E.copy()

In [0]:
aInput = x_train[0]
aInput = aInput.reshape((1, 256, 256, 3))

e0 = np.subtract(x_train[0], x_train[1])
e1 = np.subtract(x_train[1], x_train[0])
anE = np.concatenate([e0, e1], axis= -1)
anE = np.maximum(anE, 0)
anE = np.expand_dims(anE, 0)
anE = np.expand_dims(anE, 0)

for img in x_train:  
  img = img.reshape((1, 256, 256, 3))
  anE = hm.predict({'inputs': img, 'e_in': anE})
  pred = pm.predict({'inputs': img, 'e_in':anE})
  
  fig, ax = plt.subplots(1, 2)
  ax[0].set_axis_off()
  ax[0].imshow(img.reshape((256, 256, 3)))
  
  ax[1].set_axis_off()
  ax[1].imshow(pred.reshape((256, 256, 3)))
  plt.show()
